# Jointure NAN Values

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [3]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experiment: ExperimentConfig = configuration.experiments(0)

lastException = null
args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@3a307260
session = org.apache.spark.sql.SparkSession@3a307260
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output...

In [4]:
val joinedDFPath = s"${configuration.common.output.basePath}/Experience-20-D2_60-weatherDepthHours_3_0/data/joined_flights_weather.parquet"
val joinedDF = spark.read.parquet(joinedDFPath)

joinedDF.show(1, 1000, true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

joinedDFPath = /home/jovyan/work/output/Experience-20-D2_60-weatherDepthHours_3_0/data/joined_flights_weather.parquet
joinedDF = [CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 16 more fields]


[CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 16 more fields]

In [5]:
val joinedExplodedDFPath = s"${configuration.common.output.basePath}/Experience-20-D2_60-weatherDepthHours_3_0/data/joined_exploded_data.parquet"
val joinedExplodedDF = spark.read.parquet(joinedExplodedDFPath)

joinedExplodedDF.show(1, 1000, true)

-RECORD 0----------------------------------------------------------------
 CRS_DEP_TIME                                       | 2120               
 CRS_ELAPSED_TIME                                   | 95.0               
 DEST_AIRPORT_ID                                    | 11259              
 DEST_WBAN                                          | 13960              
 OP_CARRIER_AIRLINE_ID                              | 19393              
 ORIGIN_AIRPORT_ID                                  | 13198              
 ORIGIN_WBAN                                        | 03947              
 PREV_AIRCRAFT_ARR_DELAY                            | 0.0                
 UTC_FL_DATE                                        | 2012-01-20         
 feature_arrival_time_period                        | Night              
 feature_avg_delay                                  | 14.352941176470589 
 feature_departure_hour_rounded                     | 2100               
 feature_departure_time_period        

joinedExplodedDFPath = /home/jovyan/work/output/Experience-20-D2_60-weatherDepthHours_3_0/data/joined_exploded_data.parquet
joinedExplodedDF = [CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 263 more fields]


[CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 263 more fields]

In [6]:
print(joinedExplodedDFPath)

/home/jovyan/work/output/Experience-20-D2_60-weatherDepthHours_3_0/data/joined_exploded_data.parquet

In [7]:
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

case class ColNullNaNStat(
  column: String,
  null_count: Long,
  nan_count: Long,
  total: Long,
  null_pct: Double,
  nan_pct: Double
)

def nullNanReport(df: DataFrame)(implicit spark: SparkSession): DataFrame = {
  val total = df.count()

  // Construire les agrégations NULL + NaN (NaN seulement pour Float/Double)
  val aggExprs = df.schema.fields.flatMap { f =>
    val c = col(f.name)
    val nullAgg = sum(when(c.isNull, 1).otherwise(0)).alias(s"${f.name}__nulls")
    val nanAgg =
      if (f.dataType == DoubleType || f.dataType == FloatType)
        Some(sum(when(isnan(c), 1).otherwise(0)).alias(s"${f.name}__nans"))
      else None
    nanAgg.toSeq :+ nullAgg
  }

  val countsRow = df.agg(aggExprs.head, aggExprs.tail: _*).first
  val countFieldNames = countsRow.schema.fieldNames.toSet

  import spark.implicits._
  val stats = df.schema.fields.map { f =>
    val name = f.name
    val nulls = countsRow.getAs[Long](s"${name}__nulls")
    val nans =
      if (countFieldNames.contains(s"${name}__nans"))
        countsRow.getAs[Long](s"${name}__nans")
      else 0L

    ColNullNaNStat(
      column     = name,
      null_count = nulls,
      nan_count  = nans,
      total      = total,
      null_pct   = if (total == 0) 0.0 else nulls * 100.0 / total,
      nan_pct    = if (total == 0) 0.0 else nans  * 100.0 / total
    )
  }.toSeq.toDS()

  stats.orderBy(desc("null_count"), desc("nan_count")).toDF()
}

val report = nullNanReport(joinedExplodedDF)
report.show(200, truncate = false)

defined class ColNullNaNStat
report = [column: string, null_count: bigint ... 4 more fields]


nullNanReport: (df: org.apache.spark.sql.DataFrame)(implicit spark: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


+--------------------------------------------------+----------+---------+------+--------------------+-------+
|column                                            |null_count|nan_count|total |null_pct            |nan_pct|
+--------------------------------------------------+----------+---------+------+--------------------+-------+
|origin_weather_HourlyPrecip-3                     |504171    |0        |531618|94.8370822658375    |0.0    |
|origin_weather_HourlyPrecip-2                     |502654    |0        |531618|94.55172699193783   |0.0    |
|origin_weather_HourlyPrecip-0                     |502464    |0        |531618|94.51598704332811   |0.0    |
|origin_weather_HourlyPrecip-1                     |501887    |0        |531618|94.40745046255017   |0.0    |
|origin_weather_ValueForWindCharacter-0            |453267    |0        |531618|85.26178571831653   |0.0    |
|origin_weather_ValueForWindCharacter-2            |451617    |0        |531618|84.95141248039006   |0.0    |
|origin_we

[column: string, null_count: bigint ... 4 more fields]

In [8]:
joinedExplodedDF.select("origin_weather_HourlyPrecip-3").distinct().show()

+-----------------------------+
|origin_weather_HourlyPrecip-3|
+-----------------------------+
|                          0.0|
|                         0.05|
|                          0.1|
|                         0.09|
|                         0.04|
|                         0.14|
|                         0.08|
|                         0.15|
|                         0.03|
|                         0.11|
|                         0.06|
|                         0.13|
|                         0.07|
|                         0.17|
|                         1.31|
|                         0.18|
|                         0.19|
|                         0.23|
|                         NULL|
|                         0.01|
+-----------------------------+
only showing top 20 rows



In [14]:
import org.apache.spark.sql.functions._

// dfReport = ton DataFrame du tableau ci-dessus (avec la colonne "column")
val featuresDistinct = report
  .select(
    regexp_replace(                             // enlève le suffixe -N
      regexp_replace(                           // enlève le préfixe origin_weather_feature_
        regexp_replace(col("column"), "^origin_weather_feature_", ""),
        "^origin_weather_", ""                  // puis origin_weather_
      ),
      "-\\d+$", ""                              // retire -0 / -1 / -2 / -3 à la fin
    ).alias("feature_base")
  )
  .distinct()
  .orderBy("feature_base")

featuresDistinct.show(truncate = false)

+-----------------------+
|feature_base           |
+-----------------------+
|Altimeter              |
|CRS_DEP_TIME           |
|CRS_ELAPSED_TIME       |
|DEST_AIRPORT_ID        |
|DEST_WBAN              |
|DewPointCelsius        |
|DewPointFarenheit      |
|DryBulbCelsius         |
|DryBulbFarenheit       |
|HourlyPrecip           |
|Icing_Risk_Flag        |
|Icing_Risk_Level       |
|OP_CARRIER_AIRLINE_ID  |
|ORIGIN_AIRPORT_ID      |
|ORIGIN_WBAN            |
|PREV_AIRCRAFT_ARR_DELAY|
|PressureChange         |
|PressureTendency       |
|RelativeHumidity       |
|SeaLevelPressure       |
+-----------------------+
only showing top 20 rows



featuresDistinct = [feature_base: string]


[feature_base: string]

## Flights NAN

In [15]:
val rawFlightsDFPath = s"${configuration.common.output.basePath}/common/data/raw_flights.parquet"
val rawFlightsDF = spark.read.parquet(rawFlightsDFPath)


rawFlightsDFPath = /home/jovyan/work/output/common/data/raw_flights.parquet
rawFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]

In [16]:
val rawFlightReport = nullNanReport(rawFlightsDF)
rawFlightReport.show(200, truncate = false)

rawFlightReport = [column: string, null_count: bigint ... 4 more fields]


+---------------------+----------+---------+------+------------------+-------+
|column               |null_count|nan_count|total |null_pct          |nan_pct|
+---------------------+----------+---------+------+------------------+-------+
|CANCELLED            |486133    |0        |486133|100.0             |0.0    |
|DIVERTED             |486133    |0        |486133|100.0             |0.0    |
|WEATHER_DELAY        |415225    |0        |486133|85.4138682212481  |0.0    |
|NAS_DELAY            |415225    |0        |486133|85.4138682212481  |0.0    |
|ARR_DELAY_NEW        |8108      |0        |486133|1.6678563273836584|0.0    |
|FL_DATE              |0         |0        |486133|0.0               |0.0    |
|OP_CARRIER_AIRLINE_ID|0         |0        |486133|0.0               |0.0    |
|OP_CARRIER_FL_NUM    |0         |0        |486133|0.0               |0.0    |
|ORIGIN_AIRPORT_ID    |0         |0        |486133|0.0               |0.0    |
|DEST_AIRPORT_ID      |0         |0        |486133|0

[column: string, null_count: bigint ... 4 more fields]

In [17]:
val porcessedFlightsDFPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val porcessedFlightsDF = spark.read.parquet(porcessedFlightsDFPath)

porcessedFlightsDFPath = /home/jovyan/work/output/common/data/processed_flights.parquet
porcessedFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 135 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 135 more fields]

In [18]:
val processedFlightReport = nullNanReport(porcessedFlightsDF)
processedFlightReport.show(200, truncate = false)

processedFlightReport = [column: string, null_count: bigint ... 4 more fields]


+----------------------------------+----------+---------+------+------------------+-------+
|column                            |null_count|nan_count|total |null_pct          |nan_pct|
+----------------------------------+----------+---------+------+------------------+-------+
|WEATHER_DELAY                     |118446    |0        |138506|85.51687291525276 |0.0    |
|NAS_DELAY                         |118446    |0        |138506|85.51687291525276 |0.0    |
|ARR_DELAY_NEW                     |2319      |0        |138506|1.6742956983812975|0.0    |
|FL_DATE                           |0         |0        |138506|0.0               |0.0    |
|OP_CARRIER_AIRLINE_ID             |0         |0        |138506|0.0               |0.0    |
|OP_CARRIER_FL_NUM                 |0         |0        |138506|0.0               |0.0    |
|ORIGIN_AIRPORT_ID                 |0         |0        |138506|0.0               |0.0    |
|DEST_AIRPORT_ID                   |0         |0        |138506|0.0             

[column: string, null_count: bigint ... 4 more fields]

In [19]:
val joinedDFReport = nullNanReport(joinedDF)
joinedDFReport.show(200, truncate = false)

joinedDFReport = [column: string, null_count: bigint ... 4 more fields]


+----------------------------------+----------+---------+------+--------+-------+
|column                            |null_count|nan_count|total |null_pct|nan_pct|
+----------------------------------+----------+---------+------+--------+-------+
|CRS_DEP_TIME                      |0         |0        |141717|0.0     |0.0    |
|CRS_ELAPSED_TIME                  |0         |0        |141717|0.0     |0.0    |
|DEST_AIRPORT_ID                   |0         |0        |141717|0.0     |0.0    |
|DEST_WBAN                         |0         |0        |141717|0.0     |0.0    |
|OP_CARRIER_AIRLINE_ID             |0         |0        |141717|0.0     |0.0    |
|ORIGIN_AIRPORT_ID                 |0         |0        |141717|0.0     |0.0    |
|ORIGIN_WBAN                       |0         |0        |141717|0.0     |0.0    |
|PREV_AIRCRAFT_ARR_DELAY           |0         |0        |141717|0.0     |0.0    |
|UTC_FL_DATE                       |0         |0        |141717|0.0     |0.0    |
|feature_arrival

[column: string, null_count: bigint ... 4 more fields]

In [25]:
print("Row Flights count = ", rawFlightsDF.count())
print("Processed Flights count = ", porcessedFlightsDF.count())
print("Joinned Flights count = ", joinedDF.count())
print("Joinned Exploded Flights count = ", joinedExplodedDF.count())

(Row Flights count = ,486133)(Processed Flights count = ,138506)(Joinned Flights count = ,141717)(Joinned Exploded Flights count = ,141717)

## Probleme Null/NAN dans Flight vient de l'explosion des colonnes weather apres jointure

In [ ]:
import com.flightdelay.features.FeaturePipeline

val explodedDF = FeaturePipeline.explose(joinedDF, experiment)
